In [1]:
import os
import pandas as pd
from collections import Counter

lisa_path = r'traffic_lights'  # Your LISA folder

print("="*80)
print("🚥 COMPLETE LISA TRAFFIC LIGHT DATASET STRUCTURE ANALYSIS")
print("="*80)

# ===== PART 1: FOLDER STRUCTURE =====
print("\n📁 PART 1: FOLDER STRUCTURE")
print("-"*80)

def explore_structure(path, level=0, max_level=4):
    """Recursively explore folder structure"""
    if level > max_level:
        return
    
    try:
        items = os.listdir(path)
        for item in sorted(items)[:20]:  # Limit to 20 items per level
            item_path = os.path.join(path, item)
            indent = "  " * level
            
            if os.path.isdir(item_path):
                print(f"{indent}📂 {item}/")
                # Count files inside
                try:
                    contents = os.listdir(item_path)
                    file_count = len([f for f in contents if os.path.isfile(os.path.join(item_path, f))])
                    folder_count = len([f for f in contents if os.path.isdir(os.path.join(item_path, f))])
                    if file_count > 0 or folder_count > 0:
                        print(f"{indent}   ({folder_count} folders, {file_count} files)")
                    explore_structure(item_path, level + 1, max_level)
                except:
                    pass
            else:
                # Show file with extension
                ext = os.path.splitext(item)[1]
                print(f"{indent}📄 {item} {ext}")
    except Exception as e:
        print(f"{indent}⚠️ Error: {e}")

explore_structure(lisa_path, max_level=3)

# ===== PART 2: FIND ALL CSV ANNOTATION FILES =====
print("\n\n📊 PART 2: ANNOTATION FILES (CSV)")
print("-"*80)

csv_files = []
for root, dirs, files in os.walk(os.path.join(lisa_path, 'Annotations')):
    for file in files:
        if file.endswith('.csv'):
            csv_path = os.path.join(root, file)
            csv_files.append(csv_path)
            file_size = os.path.getsize(csv_path) / 1024  # KB
            print(f"📄 {file}")
            print(f"   Path: {csv_path}")
            print(f"   Size: {file_size:.1f} KB")

print(f"\n✅ Total CSV files found: {len(csv_files)}")

# ===== PART 3: ANALYZE CSV CONTENT =====
print("\n\n📋 PART 3: CSV FILE CONTENT ANALYSIS")
print("-"*80)

if csv_files:
    # Read first CSV to see format
    first_csv = csv_files[0]
    print(f"\n🔍 Analyzing: {os.path.basename(first_csv)}")
    print("-"*80)
    
    try:
        # Try different separators
        for sep in [';', ',', '\t', '|']:
            try:
                df = pd.read_csv(first_csv, sep=sep, nrows=5)
                if len(df.columns) > 1:
                    print(f"✅ Separator: '{sep}'")
                    print(f"✅ Columns: {list(df.columns)}")
                    print(f"\n📊 First 3 rows:")
                    print(df.head(3).to_string())
                    
                    # Read full file for stats
                    df_full = pd.read_csv(first_csv, sep=sep)
                    print(f"\n📈 Statistics for {os.path.basename(first_csv)}:")
                    print(f"   Total rows: {len(df_full)}")
                    
                    # Try to find label/annotation column
                    for col in df_full.columns:
                        if 'tag' in col.lower() or 'label' in col.lower() or 'class' in col.lower():
                            print(f"\n🏷️  Label column: '{col}'")
                            print(f"   Unique labels: {df_full[col].unique()}")
                            print(f"   Label distribution:")
                            label_counts = df_full[col].value_counts()
                            for label, count in label_counts.items():
                                print(f"      {label}: {count}")
                    break
            except:
                continue
    except Exception as e:
        print(f"⚠️ Error reading CSV: {e}")

# ===== PART 4: FIND ALL IMAGES =====
print("\n\n🖼️  PART 4: IMAGE FILES ANALYSIS")
print("-"*80)

image_folders = []
image_stats = {}

for folder in ['dayTrain', 'nightTrain', 'daySequence1', 'daySequence2', 
               'nightSequence1', 'nightSequence2', 'sample-dayClip6', 'sample-nightClip1']:
    folder_path = os.path.join(lisa_path, folder)
    if os.path.exists(folder_path):
        # Look for 'frames' subfolder
        frames_path = os.path.join(folder_path, folder, 'frames')
        if os.path.exists(frames_path):
            images = [f for f in os.listdir(frames_path) if f.endswith(('.jpg', '.png', '.jpeg'))]
            image_stats[folder] = len(images)
            print(f"📂 {folder}/")
            print(f"   Path: {frames_path}")
            print(f"   Images: {len(images)}")
            if images:
                print(f"   Sample: {images[0]}")

total_images = sum(image_stats.values())
print(f"\n✅ Total images found: {total_images}")

# ===== PART 5: SAMPLE DATA MATCHING =====
print("\n\n🔗 PART 5: ANNOTATION-TO-IMAGE MATCHING")
print("-"*80)

if csv_files:
    print("Checking if image filenames in CSV match actual image files...")
    
    try:
        df = pd.read_csv(csv_files[0], sep=';')
        filename_col = [col for col in df.columns if 'filename' in col.lower()][0]
        sample_filenames = df[filename_col].head(5).tolist()
        
        print(f"\nSample filenames from CSV:")
        for fname in sample_filenames:
            print(f"  • {fname}")
            
            # Try to find this image
            found = False
            for folder, count in image_stats.items():
                frames_path = os.path.join(lisa_path, folder, folder, 'frames')
                if os.path.exists(os.path.join(frames_path, fname)):
                    print(f"    ✅ Found in: {folder}/frames/")
                    found = True
                    break
            
            if not found:
                print(f"    ⚠️ Image not found!")
    except Exception as e:
        print(f"⚠️ Error: {e}")

# ===== FINAL SUMMARY =====
print("\n\n" + "="*80)
print("📊 DATASET SUMMARY")
print("="*80)
print(f"✅ CSV annotation files: {len(csv_files)}")
print(f"✅ Image folders: {len(image_stats)}")
print(f"✅ Total images: {total_images}")
print("\n🎯 Ready for YOLO conversion!")
print("="*80)


🚥 COMPLETE LISA TRAFFIC LIGHT DATASET STRUCTURE ANALYSIS

📁 PART 1: FOLDER STRUCTURE
--------------------------------------------------------------------------------
📂 Annotations/
   (1 folders, 0 files)
  📂 Annotations/
     (6 folders, 0 files)
    📂 daySequence1/
       (0 folders, 2 files)
      📄 frameAnnotationsBOX.csv .csv
      📄 frameAnnotationsBULB.csv .csv
    📂 daySequence2/
       (0 folders, 2 files)
      📄 frameAnnotationsBOX.csv .csv
      📄 frameAnnotationsBULB.csv .csv
    📂 dayTrain/
       (13 folders, 0 files)
      📂 dayClip1/
         (0 folders, 2 files)
      📂 dayClip10/
         (0 folders, 2 files)
      📂 dayClip11/
         (0 folders, 2 files)
      📂 dayClip12/
         (0 folders, 2 files)
      📂 dayClip13/
         (0 folders, 2 files)
      📂 dayClip2/
         (0 folders, 2 files)
      📂 dayClip3/
         (0 folders, 2 files)
      📂 dayClip4/
         (0 folders, 2 files)
      📂 dayClip5/
         (0 folders, 2 files)
      📂 dayClip6/
       

In [2]:
import os
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split
import shutil
import yaml
from tqdm import tqdm
from collections import defaultdict

# ===== PATHS =====
lisa_path = r'traffic_lights'
output_path = r'traffic_lights_yolo'

print("="*80)
print("🚥 LISA TRAFFIC LIGHT DATASET → YOLO FORMAT CONVERSION")
print("="*80)

# Create output structure
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(output_path, split, 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_path, split, 'labels'), exist_ok=True)

# ===== CLASS MAPPING =====
class_mapping = {
    'go': 0,           # Green light
    'goLeft': 1,       # Green left arrow
    'warning': 2,      # Yellow light
    'warningLeft': 3,  # Yellow left arrow
    'stop': 4,         # Red light
    'stopLeft': 5,     # Red left arrow
}

class_names = ['green', 'green_left', 'yellow', 'yellow_left', 'red', 'red_left']

print("\n📊 Traffic Light Classes:")
for lisa_label, class_id in class_mapping.items():
    print(f"  {class_id}: {class_names[class_id]:15} (LISA: '{lisa_label}')")
print()

# ===== FIND ALL CSV FILES =====
annotations_base = os.path.join(lisa_path, 'Annotations', 'Annotations')
csv_files = []

for root, dirs, files in os.walk(annotations_base):
    for file in files:
        if file == 'frameAnnotationsBOX.csv':  # Use BOX not BULB
            csv_files.append(os.path.join(root, file))

print(f"✅ Found {len(csv_files)} annotation files (BOX format)\n")

# ===== READ ALL ANNOTATIONS =====
all_annotations = defaultdict(list)  # filename -> list of annotations

for csv_file in tqdm(csv_files, desc="📖 Reading CSV files"):
    try:
        df = pd.read_csv(csv_file, sep=';')
        
        for _, row in df.iterrows():
            # CSV format
            filename_in_csv = row['Filename']  # e.g., "dayTest/daySequence1--00000.jpg"
            label = row['Annotation tag']
            x1 = int(row['Upper left corner X'])
            y1 = int(row['Upper left corner Y'])
            x2 = int(row['Lower right corner X'])
            y2 = int(row['Lower right corner Y'])
            
            # Skip unknown labels
            if label not in class_mapping:
                continue
            
            # Extract just the image filename (remove path prefix)
            actual_filename = os.path.basename(filename_in_csv)
            
            all_annotations[actual_filename].append({
                'label': label,
                'x1': x1,
                'y1': y1,
                'x2': x2,
                'y2': y2
            })
    except Exception as e:
        print(f"  ⚠️ Error reading {csv_file}: {e}")
        continue

print(f"\n✅ Total annotated images: {len(all_annotations)}")
print(f"✅ Total annotations: {sum(len(v) for v in all_annotations.values())}")

# ===== FIND IMAGE FILES =====
image_locations = {}  # filename -> full path

# All possible image locations
search_folders = [
    ('dayTrain', ['dayClip1', 'dayClip2', 'dayClip3', 'dayClip4', 'dayClip5', 
                  'dayClip6', 'dayClip7', 'dayClip8', 'dayClip9', 'dayClip10', 
                  'dayClip11', 'dayClip12', 'dayClip13']),
    ('nightTrain', ['nightClip1', 'nightClip2', 'nightClip3', 'nightClip4', 'nightClip5']),
    ('daySequence1', ['']),
    ('daySequence2', ['']),
    ('nightSequence1', ['']),
    ('nightSequence2', ['']),
    ('sample-dayClip6', ['']),
    ('sample-nightClip1', [''])
]

print("\n🔍 Mapping image files to paths...")
for base_folder, subfolders in search_folders:
    if not subfolders or subfolders == ['']:
        # Direct folder (like daySequence1)
        frames_path = os.path.join(lisa_path, base_folder, base_folder, 'frames')
        if os.path.exists(frames_path):
            for img_file in os.listdir(frames_path):
                if img_file.endswith('.jpg'):
                    image_locations[img_file] = os.path.join(frames_path, img_file)
    else:
        # Has subfolders (like dayTrain/dayClip1)
        for subfolder in subfolders:
            frames_path = os.path.join(lisa_path, base_folder, base_folder, subfolder, 'frames')
            if os.path.exists(frames_path):
                for img_file in os.listdir(frames_path):
                    if img_file.endswith('.jpg'):
                        image_locations[img_file] = os.path.join(frames_path, img_file)

print(f"✅ Found {len(image_locations)} image files")

# ===== MATCH ANNOTATIONS TO IMAGES =====
matched_data = []

for img_filename, annotations in all_annotations.items():
    if img_filename in image_locations:
        matched_data.append({
            'filename': img_filename,
            'image_path': image_locations[img_filename],
            'annotations': annotations
        })

print(f"✅ Matched {len(matched_data)} images with annotations\n")

# ===== SPLIT DATASET =====
train_data, temp_data = train_test_split(matched_data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

print(f"📦 Dataset Split:")
print(f"  Train: {len(train_data):5,} images")
print(f"  Val:   {len(val_data):5,} images")
print(f"  Test:  {len(test_data):5,} images")
print(f"  Total: {len(matched_data):5,} images\n")

# ===== CONVERT TO YOLO FORMAT =====
def convert_to_yolo(img_width, img_height, x1, y1, x2, y2):
    """Convert bbox to YOLO format"""
    x_center = ((x1 + x2) / 2) / img_width
    y_center = ((y1 + y2) / 2) / img_height
    width = (x2 - x1) / img_width
    height = (y2 - y1) / img_height
    return x_center, y_center, width, height

def process_split(data_list, split_name):
    """Process and save images/labels for a split"""
    processed = 0
    
    for idx, item in enumerate(tqdm(data_list, desc=f"🔄 {split_name:5}")):
        try:
            # Read image
            img = cv2.imread(item['image_path'])
            if img is None:
                continue
            
            img_height, img_width = img.shape[:2]
            
            # Save image
            new_img_name = f"{split_name}_{processed:06d}.jpg"
            cv2.imwrite(os.path.join(output_path, split_name, 'images', new_img_name), img)
            
            # Create YOLO label
            label_path = os.path.join(output_path, split_name, 'labels', 
                                     new_img_name.replace('.jpg', '.txt'))
            
            with open(label_path, 'w') as f:
                for ann in item['annotations']:
                    class_id = class_mapping[ann['label']]
                    x_c, y_c, w, h = convert_to_yolo(
                        img_width, img_height, ann['x1'], ann['y1'], ann['x2'], ann['y2']
                    )
                    f.write(f"{class_id} {x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}\n")
            
            processed += 1
            
        except Exception as e:
            continue
    
    return processed

# Process all splits
print("🔄 Converting to YOLO format...\n")
train_count = process_split(train_data, 'train')
val_count = process_split(val_data, 'val')
test_count = process_split(test_data, 'test')

# ===== CREATE DATA.YAML =====
data_yaml = {
    'path': output_path,
    'train': 'train/images',
    'val': 'val/images',
    'test': 'test/images',
    'nc': len(class_names),
    'names': class_names
}

yaml_path = os.path.join(output_path, 'data.yaml')
with open(yaml_path, 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

# ===== FINAL SUMMARY =====
print("\n" + "="*80)
print("🎉 LISA DATASET CONVERSION COMPLETE!")
print("="*80)
print(f"\n📍 Output Location: {output_path}")
print(f"\n📊 Final Statistics:")
print(f"  Train: {train_count:5,} images")
print(f"  Val:   {val_count:5,} images")
print(f"  Test:  {test_count:5,} images")
print(f"  {'─'*40}")
print(f"  TOTAL: {train_count + val_count + test_count:5,} images")

print(f"\n🚦 Classes ({len(class_names)} types):")
for idx, name in enumerate(class_names):
    print(f"  {idx}: {name}")

print(f"\n✅ data.yaml created!")
print(f"✅ Ready to merge with KITTI + GTSRB!")
print("="*80)


🚥 LISA TRAFFIC LIGHT DATASET → YOLO FORMAT CONVERSION

📊 Traffic Light Classes:
  0: green           (LISA: 'go')
  1: green_left      (LISA: 'goLeft')
  2: yellow          (LISA: 'warning')
  3: yellow_left     (LISA: 'warningLeft')
  4: red             (LISA: 'stop')
  5: red_left        (LISA: 'stopLeft')

✅ Found 22 annotation files (BOX format)



📖 Reading CSV files: 100%|██████████| 22/22 [00:05<00:00,  4.37it/s]



✅ Total annotated images: 36195
✅ Total annotations: 109270

🔍 Mapping image files to paths...
✅ Found 43016 image files
✅ Matched 36195 images with annotations

📦 Dataset Split:
  Train: 25,336 images
  Val:   5,429 images
  Test:  5,430 images
  Total: 36,195 images

🔄 Converting to YOLO format...



🔄 test : 100%|██████████| 5430/5430 [04:02<00:00, 22.38it/s]


🎉 LISA DATASET CONVERSION COMPLETE!

📍 Output Location: traffic_lights_yolo

📊 Final Statistics:
  Train: 25,336 images
  Val:   5,429 images
  Test:  5,430 images
  ────────────────────────────────────────
  TOTAL: 36,195 images

🚦 Classes (6 types):
  0: green
  1: green_left
  2: yellow
  3: yellow_left
  4: red
  5: red_left

✅ data.yaml created!
✅ Ready to merge with KITTI + GTSRB!


In [1]:
import os
import shutil
import yaml
from tqdm import tqdm

print("="*80)
print("🚗🚦🚥 ULTIMATE AUTONOMOUS VEHICLE DATASET MERGE")
print("="*80)
print("Combining: KITTI (Vehicles) + GTSRB (Signs) + LISA (Lights)")
print("="*80)

# ===== PATHS =====
kitti_path = r'D:\Depi\Graduation Project\yolo_dataset'
signs_path = r'traffic_signs'
lights_path = r'traffic_lights_yolo'
output_path = r'D:\Depi\Graduation Project\ultimate_autonomous_dataset'

# ===== DATASET 1: KITTI - VEHICLES (IDs 0-7) =====
kitti_classes = [
    'Car', 'Van', 'Truck', 'Pedestrian', 
    'Person_sitting', 'Cyclist', 'Tram', 'Misc'
]

# ===== DATASET 2: GTSRB - TRAFFIC SIGNS (IDs 8-50) =====
sign_classes = [
    '20_speed', '30_speed', '50_speed', '60_speed', '70_speed', '80_speed',
    '80_lifted', '100_speed', '120_speed', 'no_overtaking_general',
    'no_overtaking_trucks', 'right_of_way_crossing', 'right_of_way_general',
    'give_way', 'stop', 'no_way_general', 'no_way_trucks', 'no_way_one_way',
    'attention_general', 'attention_left_turn', 'attention_right_turn',
    'attention_curvy', 'attention_bumpers', 'attention_slippery',
    'attention_bottleneck', 'attention_construction', 'attention_traffic_light',
    'attention_pedestrian', 'attention_children', 'attention_bikes',
    'attention_snowflake', 'attention_deer', 'lifted_general', 'turn_right',
    'turn_left', 'turn_straight', 'turn_straight_right', 'turn_straight_left',
    'turn_right_down', 'turn_left_down', 'turn_circle',
    'lifted_no_overtaking_general', 'lifted_no_overtaking_trucks'
]

# ===== DATASET 3: LISA - TRAFFIC LIGHTS (IDs 51-56) =====
light_classes = [
    'traffic_light_green',
    'traffic_light_green_left',
    'traffic_light_yellow',
    'traffic_light_yellow_left',
    'traffic_light_red',
    'traffic_light_red_left'
]

# ===== COMBINED CLASSES =====
combined_classes = kitti_classes + sign_classes + light_classes

offset_kitti = 0
offset_signs = len(kitti_classes)
offset_lights = len(kitti_classes) + len(sign_classes)

print(f"\n📊 COMPLETE AUTONOMOUS VEHICLE PERCEPTION SYSTEM:")
print(f"{'='*80}")
print(f"  🚗 Vehicles (KITTI):       {len(kitti_classes):2} classes  [IDs {offset_kitti:2}-{offset_signs-1:2}]")
print(f"  🚦 Traffic Signs (GTSRB):  {len(sign_classes):2} classes  [IDs {offset_signs:2}-{offset_lights-1:2}]")
print(f"  🚥 Traffic Lights (LISA):  {len(light_classes):2} classes  [IDs {offset_lights:2}-{offset_lights+len(light_classes)-1:2}]")
print(f"{'='*80}")
print(f"  TOTAL CLASSES: {len(combined_classes)}")
print()

# Create output structure
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(output_path, split, 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_path, split, 'labels'), exist_ok=True)

def copy_dataset(source, dest, split, offset, prefix):
    """Copy dataset and remap class IDs"""
    src_img = os.path.join(source, split, 'images')
    src_lbl = os.path.join(source, split, 'labels')
    dst_img = os.path.join(dest, split, 'images')
    dst_lbl = os.path.join(dest, split, 'labels')
    
    if not os.path.exists(src_img):
        print(f"      ⚠️ {split} not found, skipping...")
        return 0
    
    images = [f for f in os.listdir(src_img) if f.endswith(('.png', '.jpg'))]
    
    for img in tqdm(images, desc=f"      {split:5}"):
        try:
            # Copy image with prefix
            new_img = f"{prefix}_{img}"
            shutil.copy(os.path.join(src_img, img), os.path.join(dst_img, new_img))
            
            # Copy and remap label
            lbl = img.replace('.png', '.txt').replace('.jpg', '.txt')
            src = os.path.join(src_lbl, lbl)
            dst = os.path.join(dst_lbl, f"{prefix}_{lbl}")
            
            if os.path.exists(src):
                with open(src, 'r') as f:
                    lines = f.readlines()
                with open(dst, 'w') as f:
                    for line in lines:
                        parts = line.strip().split()
                        if len(parts) >= 5:
                            new_cls = int(parts[0]) + offset
                            f.write(f"{new_cls} {' '.join(parts[1:])}\n")
        except Exception as e:
            continue
    
    return len(images)

# ===== MERGE DATASETS =====
print("\n📦 STEP 1/3: Copying KITTI (Vehicles)...")
kitti_counts = {}
for split in ['train', 'val', 'test']:
    count = copy_dataset(kitti_path, output_path, split, offset_kitti, 'kitti')
    kitti_counts[split] = count
print(f"   ✅ KITTI Total: {sum(kitti_counts.values()):,} images\n")

print("🚦 STEP 2/3: Copying GTSRB (Traffic Signs)...")
sign_counts = {}
for split in ['train', 'test']:
    count = copy_dataset(signs_path, output_path, split, offset_signs, 'signs')
    sign_counts[split] = count
print(f"   ✅ GTSRB Total: {sum(sign_counts.values()):,} images\n")

print("🚥 STEP 3/3: Copying LISA (Traffic Lights)...")
light_counts = {}
for split in ['train', 'val', 'test']:
    count = copy_dataset(lights_path, output_path, split, offset_lights, 'lights')
    light_counts[split] = count
print(f"   ✅ LISA Total: {sum(light_counts.values()):,} images\n")

# ===== CREATE DATA.YAML =====
data_yaml = {
    'path': output_path,
    'train': 'train/images',
    'val': 'val/images',
    'test': 'test/images',
    'nc': len(combined_classes),
    'names': combined_classes
}

yaml_path = os.path.join(output_path, 'data.yaml')
with open(yaml_path, 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

# ===== FINAL STATISTICS =====
total_train = kitti_counts.get('train', 0) + sign_counts.get('train', 0) + light_counts.get('train', 0)
total_val = kitti_counts.get('val', 0) + sign_counts.get('val', 0) + light_counts.get('val', 0)
total_test = kitti_counts.get('test', 0) + sign_counts.get('test', 0) + light_counts.get('test', 0)
grand_total = total_train + total_val + total_test

print("="*80)
print("🎉 ULTIMATE AUTONOMOUS VEHICLE DATASET COMPLETE!")
print("="*80)
print(f"\n📍 Location: {output_path}")

print(f"\n📊 FINAL DATASET STATISTICS:")
print(f"{'─'*80}")
print(f"  Train:  {total_train:7,} images")
print(f"  Val:    {total_val:7,} images")
print(f"  Test:   {total_test:7,} images")
print(f"  {'─'*80}")
print(f"  TOTAL:  {grand_total:7,} IMAGES")
print(f"{'─'*80}")

print(f"\n🎯 PERCEPTION CAPABILITIES:")
print(f"  🚗 Vehicles:       {len(kitti_classes):2} types  (Car, Truck, Pedestrian, Cyclist, etc.)")
print(f"  🚦 Traffic Signs:  {len(sign_classes):2} types  (Speed limits, Stop, Yield, Warnings, etc.)")
print(f"  🚥 Traffic Lights: {len(light_classes):2} types  (Red, Yellow, Green + Arrows)")
print(f"  {'─'*80}")
print(f"  TOTAL:             {len(combined_classes):2} CLASSES")

print(f"\n✅ data.yaml created at: {yaml_path}")
print(f"\n🚀 NEXT STEP: Train your unified autonomous vehicle model!")
print("="*80)


🚗🚦🚥 ULTIMATE AUTONOMOUS VEHICLE DATASET MERGE
Combining: KITTI (Vehicles) + GTSRB (Signs) + LISA (Lights)

📊 COMPLETE AUTONOMOUS VEHICLE PERCEPTION SYSTEM:
  🚗 Vehicles (KITTI):        8 classes  [IDs  0- 7]
  🚦 Traffic Signs (GTSRB):  43 classes  [IDs  8-50]
  🚥 Traffic Lights (LISA):   6 classes  [IDs 51-56]
  TOTAL CLASSES: 57


📦 STEP 1/3: Copying KITTI (Vehicles)...


      test : 100%|██████████| 1123/1123 [00:29<00:00, 38.40it/s]


   ✅ KITTI Total: 7,481 images

🚦 STEP 2/3: Copying GTSRB (Traffic Signs)...


      test : 100%|██████████| 12630/12630 [05:47<00:00, 36.31it/s]


   ✅ GTSRB Total: 51,839 images

🚥 STEP 3/3: Copying LISA (Traffic Lights)...


      test : 100%|██████████| 5430/5430 [02:14<00:00, 40.44it/s]

   ✅ LISA Total: 36,195 images

🎉 ULTIMATE AUTONOMOUS VEHICLE DATASET COMPLETE!

📍 Location: D:\Depi\Graduation Project\ultimate_autonomous_dataset

📊 FINAL DATASET STATISTICS:
────────────────────────────────────────────────────────────────────────────────
  Train:   69,781 images
  Val:      6,551 images
  Test:    19,183 images
  ────────────────────────────────────────────────────────────────────────────────
  TOTAL:   95,515 IMAGES
────────────────────────────────────────────────────────────────────────────────

🎯 PERCEPTION CAPABILITIES:
  🚗 Vehicles:        8 types  (Car, Truck, Pedestrian, Cyclist, etc.)
  🚦 Traffic Signs:  43 types  (Speed limits, Stop, Yield, Warnings, etc.)
  🚥 Traffic Lights:  6 types  (Red, Yellow, Green + Arrows)
  ────────────────────────────────────────────────────────────────────────────────
  TOTAL:             57 CLASSES

✅ data.yaml created at: D:\Depi\Graduation Project\ultimate_autonomous_dataset\data.yaml

🚀 NEXT STEP: Train your unified autonomo

In [ ]:
from ultralytics import YOLO
import torch

print("="*80)
print("🚗🚦🚥 TRAINING ULTIMATE AUTONOMOUS VEHICLE MODEL")
print("="*80)

# Check GPU
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"GPU Memory: 8GB RTX 4060 Laptop")

# Use YOLOv8s for better performance
model = YOLO('yolov8s.pt')

print("\n🎯 Training Configuration:")
print(f"  Model: YOLOv8s (11M parameters)")
print(f"  Batch Size: 32 (maximizing GPU usage)")
print(f"  Classes: 57 (8 vehicles + 43 signs + 6 lights)")
print(f"  Total Images: ~100,000+")
print(f"  Expected Training Time: 1.5-2 hours")
print()

results = model.train(
    data=r'D:\Depi\Graduation Project\ultimate_autonomous_dataset\data.yaml',
    epochs=50,
    imgsz=640,
    batch=32,  # Using more GPU!
    patience=25,
    device=0,
    workers=4,
    
    # Optimizer
    optimizer='AdamW',
    lr0=0.001,  # Lower LR for fine-tuning
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    
    # Augmentation
    mosaic=1.0,
    mixup=0.1,
    degrees=5.0,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    
    # Settings
    project=r'D:\Depi\Graduation Project\runs',
    name='ultimate_autonomous_model',
    exist_ok=False,
    verbose=True,
    amp=True,
    cache=False,
    plots=True,
    save=True,
    save_period=5
)

print("\n" + "="*80)
print("🎉 TRAINING COMPLETE!")
print("="*80)
print(f"\n📊 Results:")
print(f"  Best mAP50: {results.results_dict.get('metrics/mAP50(B)', 0):.3f}")
print(f"  Best mAP50-95: {results.results_dict.get('metrics/mAP50-95(B)', 0):.3f}")
print(f"\n💾 Model Location:")
print(f"  runs/ultimate_autonomous_model/weights/best.pt")
print(f"\n🎯 YOUR MODEL CAN NOW DETECT:")
print(f"  • 8 Vehicle types (Car, Truck, Van, Pedestrian, Cyclist, etc.)")
print(f"  • 43 Traffic sign types (All speed limits, stops, yields, warnings)")
print(f"  • 6 Traffic light states (Red, Yellow, Green + Left arrows)")
print(f"  = 57 CLASSES TOTAL - COMPLETE AUTONOMOUS DRIVING PERCEPTION!")
print("="*80)


🚗🚦🚥 TRAINING ULTIMATE AUTONOMOUS VEHICLE MODEL
GPU: NVIDIA GeForce RTX 4060 Laptop GPU
GPU Memory: 8GB RTX 4060 Laptop


100%|██████████| 21.5M/21.5M [00:05<00:00, 4.34MB/s]
c:\Users\youss\anaconda3\envs\GPU-Pytorch\lib\site-packages\ultralytics\nn\tasks.py:567: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimen


🎯 Training Configuration:
  Model: YOLOv8s (11M parameters)
  Batch Size: 32 (maximizing GPU usage)
  Classes: 57 (8 vehicles + 43 signs + 6 lights)
  Total Images: ~100,000+
  Expected Training Time: 1.5-2 hours



New https://pypi.org/project/ultralytics/8.3.220 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196  Python-3.8.20 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=D:\Depi\Graduation Project\ultimate_autonomous_dataset\data.yaml, epochs=50, patience=25, batch=32, imgsz=640, save=True, save_period=5, cache=False, device=0, workers=4, project=D:\Depi\Graduation Project\runs, name=ultimate_autonomous_model, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sho

In [ ]:
import os
import shutil
import random
from pathlib import Path

print("="*80)
print("🎯 SAMPLING 100 IMAGES FROM EACH DATASET FOR TESTING")
print("="*80)

# ===== PATHS =====
test_images_path = r'D:\Depi\Graduation Project\ultimate_autonomous_dataset\test\images'
output_sample_path = r'D:\Depi\Graduation Project\test_samples'
model_path = r'D:\Depi\Graduation Project\runs\ultimate_overnight\weights\best.pt'

# Create output folders
os.makedirs(output_sample_path, exist_ok=True)
os.makedirs(os.path.join(output_sample_path, 'kitti'), exist_ok=True)
os.makedirs(os.path.join(output_sample_path, 'signs'), exist_ok=True)
os.makedirs(os.path.join(output_sample_path, 'lights'), exist_ok=True)

# ===== STEP 1: COLLECT IMAGES BY DATASET =====
print("\n📂 Step 1: Collecting images by dataset...")

all_test_images = os.listdir(test_images_path)

# Separate by prefix (how you named them during merge)
kitti_images = [img for img in all_test_images if img.startswith('kitti_')]
signs_images = [img for img in all_test_images if img.startswith('signs_')]
lights_images = [img for img in all_test_images if img.startswith('lights_')]

print(f"✅ Found {len(kitti_images)} KITTI images")
print(f"✅ Found {len(signs_images)} Traffic Sign images")
print(f"✅ Found {len(lights_images)} Traffic Light images")

# ===== STEP 2: RANDOMLY SAMPLE 100 FROM EACH =====
print("\n🎲 Step 2: Randomly sampling 100 images from each dataset...")

random.seed(42)  # For reproducibility

sample_kitti = random.sample(kitti_images, min(100, len(kitti_images)))
sample_signs = random.sample(signs_images, min(100, len(signs_images)))
sample_lights = random.sample(lights_images, min(100, len(lights_images)))

print(f"✅ Sampled {len(sample_kitti)} KITTI images")
print(f"✅ Sampled {len(sample_signs)} Traffic Sign images")
print(f"✅ Sampled {len(sample_lights)} Traffic Light images")

# ===== STEP 3: COPY SAMPLED IMAGES =====
print("\n📋 Step 3: Copying sampled images...")

for img in sample_kitti:
    src = os.path.join(test_images_path, img)
    dst = os.path.join(output_sample_path, 'kitti', img)
    shutil.copy(src, dst)

for img in sample_signs:
    src = os.path.join(test_images_path, img)
    dst = os.path.join(output_sample_path, 'signs', img)
    shutil.copy(src, dst)

for img in sample_lights:
    src = os.path.join(test_images_path, img)
    dst = os.path.join(output_sample_path, 'lights', img)
    shutil.copy(src, dst)

print("✅ All samples copied!")

# ===== STEP 4: RUN PREDICTIONS ON EACH DATASET =====
print("\n" + "="*80)
print("🚀 Step 4: Running predictions on sampled images...")
print("="*80)

from ultralytics import YOLO

# Load your trained model
model = YOLO(model_path)

# Run predictions on KITTI samples
print("\n🚗 Running predictions on KITTI (Vehicles) samples...")
results_kitti = model.predict(
    source=os.path.join(output_sample_path, 'kitti'),
    conf=0.25,
    save=True,
    project=r'D:\Depi\Graduation Project\test_results',
    name='kitti_samples',
    exist_ok=True
)
print(f"✅ KITTI predictions saved!")

# Run predictions on Traffic Sign samples
print("\n🚦 Running predictions on Traffic Sign samples...")
results_signs = model.predict(
    source=os.path.join(output_sample_path, 'signs'),
    conf=0.25,
    save=True,
    project=r'D:\Depi\Graduation Project\test_results',
    name='signs_samples',
    exist_ok=True
)
print(f"✅ Traffic Sign predictions saved!")

# Run predictions on Traffic Light samples
print("\n🚥 Running predictions on Traffic Light samples...")
results_lights = model.predict(
    source=os.path.join(output_sample_path, 'lights'),
    conf=0.25,
    save=True,
    project=r'D:\Depi\Graduation Project\test_results',
    name='lights_samples',
    exist_ok=True
)
print(f"✅ Traffic Light predictions saved!")

# ===== STEP 5: SUMMARY =====
print("\n" + "="*80)
print("🎉 TESTING COMPLETE!")
print("="*80)

print(f"\n📊 Results Summary:")
print(f"  KITTI samples tested: {len(sample_kitti)}")
print(f"  Traffic Sign samples tested: {len(sample_signs)}")
print(f"  Traffic Light samples tested: {len(sample_lights)}")
print(f"  Total images tested: {len(sample_kitti) + len(sample_signs) + len(sample_lights)}")

print(f"\n📁 Results saved to:")
print(f"  KITTI: test_results/kitti_samples/")
print(f"  Signs: test_results/signs_samples/")
print(f"  Lights: test_results/lights_samples/")

print("\n✅ Check the folders above to see predicted images with bounding boxes!")
print("="*80)


🎯 SAMPLING 100 IMAGES FROM EACH DATASET FOR TESTING

📂 Step 1: Collecting images by dataset...
✅ Found 1123 KITTI images
✅ Found 12630 Traffic Sign images
✅ Found 5430 Traffic Light images

🎲 Step 2: Randomly sampling 100 images from each dataset...
✅ Sampled 100 KITTI images
✅ Sampled 100 Traffic Sign images
✅ Sampled 100 Traffic Light images

📋 Step 3: Copying sampled images...
✅ All samples copied!

🚀 Step 4: Running predictions on sampled images...


c:\Users\youss\anaconda3\envs\GPU-Pytorch\lib\site-packages\ultralytics\nn\tasks.py:567: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='


🚗 Running predictions on KITTI (Vehicles) samples...



image 1/100 D:\Depi\Graduation Project\test_samples\kitti\kitti_000079.png: 224x640 4 Cars, 1 Tram, 29.9ms
image 2/100 D:\Depi\Graduation Project\test_samples\kitti\kitti_000322.png: 224x640 2 Cars, 7.0ms
image 3/100 D:\Depi\Graduation Project\test_samples\kitti\kitti_000334.png: 224x640 1 Car, 1 Van, 7.0ms
image 4/100 D:\Depi\Graduation Project\test_samples\kitti\kitti_000373.png: 224x640 7 Cars, 5.5ms
image 5/100 D:\Depi\Graduation Project\test_samples\kitti\kitti_000429.png: 224x640 11 Cars, 9.3ms
image 6/100 D:\Depi\Graduation Project\test_samples\kitti\kitti_000534.png: 224x640 1 Car, 8.0ms
image 7/100 D:\Depi\Graduation Project\test_samples\kitti\kitti_000573.png: 224x640 6 Cars, 5.0ms
image 8/100 D:\Depi\Graduation Project\test_samples\kitti\kitti_000586.png: 224x640 9 Cars, 3 Vans, 7.0ms
image 9/100 D:\Depi\Graduation Project\test_samples\kitti\kitti_000742.png: 224x640 11 Cars, 11.0ms
image 10/100 D:\Depi\Graduation Project\test_samples\kitti\kitti_000842.png: 224x640 2 Cars,

✅ KITTI predictions saved!

🚦 Running predictions on Traffic Sign samples...


image 8/100 D:\Depi\Graduation Project\test_samples\signs\signs_00964.png: 640x640 1 no_overtaking_trucks, 6.0ms
image 9/100 D:\Depi\Graduation Project\test_samples\signs\signs_01049.png: 640x640 1 turn_straight, 7.0ms
image 10/100 D:\Depi\Graduation Project\test_samples\signs\signs_01104.png: 608x640 1 attention_slippery, 5.8ms
image 11/100 D:\Depi\Graduation Project\test_samples\signs\signs_01133.png: 640x640 1 70_speed, 5.0ms
image 12/100 D:\Depi\Graduation Project\test_samples\signs\signs_01163.png: 640x576 1 no_way_one_way, 29.2ms
image 13/100 D:\Depi\Graduation Project\test_samples\signs\signs_01290.png: 640x640 1 70_speed, 5.5ms
image 14/100 D:\Depi\Graduation Project\test_samples\signs\signs_01403.png: 640x640 1 50_speed, 6.2ms
image 15/100 D:\Depi\Graduation Project\test_samples\signs\signs_01743.png: 640x608 1 50_speed, 5.0ms
image 16/100 D:\Depi\Graduation Project\test_samples\signs\signs_01827.png: 576x640 1 attention_slippery, 5.3ms
image 17/100 D:\Depi\Graduation Project\

✅ Traffic Sign predictions saved!

🚥 Running predictions on Traffic Light samples...


image 4/100 D:\Depi\Graduation Project\test_samples\lights\lights_test_000410.jpg: 480x640 3 traffic_light_reds, 6.0ms
image 5/100 D:\Depi\Graduation Project\test_samples\lights\lights_test_000414.jpg: 480x640 1 traffic_light_red, 1 traffic_light_red_left, 4.2ms
image 6/100 D:\Depi\Graduation Project\test_samples\lights\lights_test_000443.jpg: 480x640 3 traffic_light_greens, 6.6ms
image 7/100 D:\Depi\Graduation Project\test_samples\lights\lights_test_000465.jpg: 480x640 1 traffic_light_red_left, 5.0ms
image 8/100 D:\Depi\Graduation Project\test_samples\lights\lights_test_000468.jpg: 480x640 3 traffic_light_greens, 2 traffic_light_red_lefts, 8.3ms
image 9/100 D:\Depi\Graduation Project\test_samples\lights\lights_test_000479.jpg: 480x640 2 traffic_light_reds, 1 traffic_light_red_left, 4.0ms
image 10/100 D:\Depi\Graduation Project\test_samples\lights\lights_test_000480.jpg: 480x640 2 traffic_light_reds, 7.0ms
image 11/100 D:\Depi\Graduation Project\test_samples\lights\lights_test_000496.j

✅ Traffic Light predictions saved!

🎉 TESTING COMPLETE!

📊 Results Summary:
  KITTI samples tested: 100
  Traffic Sign samples tested: 100
  Traffic Light samples tested: 100
  Total images tested: 300

📁 Results saved to:
  KITTI: test_results/kitti_samples/
  Signs: test_results/signs_samples/
  Lights: test_results/lights_samples/

✅ Check the folders above to see predicted images with bounding boxes!


In [5]:
from ultralytics import YOLO

print("="*80)
print("🚗 TESTING MODEL ON GERMAN DRIVING VIDEO")
print("="*80)

# Load your trained model
model_path = r'yolo_traffic_detection\best.pt'
video_path = r'Test.mp4'

print("\n🤖 Loading model...")
model = YOLO(model_path)

print("🎬 Running predictions on video...")
print("This might take a few minutes...\n")

# Run prediction
results = model.predict(
    source=video_path,
    conf=0.5,
    save=True,
    project=r'video_results',
    name='german_driving_demo',
    show_labels=True,
    show_conf=True,
    stream=True,
    verbose=True
)

# Process results
frame_count = 0
for result in results:
    frame_count += 1
    if frame_count % 100 == 0:
        print(f"✅ Processed {frame_count} frames...")

print("\n" + "="*80)
print("🎉 VIDEO PROCESSING COMPLETE!")
print("="*80)
print(f"\n📹 Output video with detections saved to:")
print(f"   video_results/german_driving_demo/german_driving_test.mp4")
print("\n✅ Open the video to see your model detecting:")
print("   🚗 Cars, Trucks, Vans")
print("   🚦 Traffic Signs")
print("   🚥 Traffic Lights")
print("="*80)


🚗 TESTING MODEL ON GERMAN DRIVING VIDEO

🤖 Loading model...
🎬 Running predictions on video...
This might take a few minutes...




video 1/1 (1/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 11.0ms
video 1/1 (2/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 11.0ms
video 1/1 (3/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 10.5ms
video 1/1 (4/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 10.4ms
video 1/1 (5/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 10.0ms
video 1/1 (6/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 11.0ms
video 1/1 (7/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 12.9ms
video 1/1 (8/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 11.0ms
video 1/1 (9/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 11.0ms
video 1/1 (10/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 11.0ms
video 1/1 (11/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 12.0ms
video 1/1 (12/4804) d:\Depi\G

✅ Processed 100 frames...


video 1/1 (106/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 12.0ms
video 1/1 (107/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 15.0ms
video 1/1 (108/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 12.5ms
video 1/1 (109/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 11.5ms
video 1/1 (110/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 22.0ms
video 1/1 (111/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 12.0ms
video 1/1 (112/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 18.0ms
video 1/1 (113/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 23.0ms
video 1/1 (114/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 41.5ms
video 1/1 (115/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 22.0ms
video 1/1 (116/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 19.5ms
video 1/1 

✅ Processed 200 frames...


...

✅ Processed 4600 frames...


video 1/1 (4606/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 13.0ms
video 1/1 (4607/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 13.7ms
video 1/1 (4608/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 13.5ms
video 1/1 (4609/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 13.4ms
video 1/1 (4610/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 13.0ms
video 1/1 (4611/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 16.5ms
video 1/1 (4612/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 14.0ms
video 1/1 (4613/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 15.0ms
video 1/1 (4614/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 15.0ms
video 1/1 (4615/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 14.5ms
video 1/1 (4616/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 13.5ms

✅ Processed 4700 frames...


video 1/1 (4706/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 10.5ms
video 1/1 (4707/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 1 Parking-Sign, 11.0ms
video 1/1 (4708/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 12.0ms
video 1/1 (4709/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 10.5ms
video 1/1 (4710/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 10.0ms
video 1/1 (4711/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 11.3ms
video 1/1 (4712/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 11.0ms
video 1/1 (4713/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 11.0ms
video 1/1 (4714/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 (no detections), 12.0ms
video 1/1 (4715/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 1 Parking-Sign, 12.0ms
video 1/1 (4716/4804) d:\Depi\Graduation Project\Test.mp4: 384x640 1 Parking-Sign, 12.0ms
vi

✅ Processed 4800 frames...

🎉 VIDEO PROCESSING COMPLETE!

📹 Output video with detections saved to:
   video_results/german_driving_demo/german_driving_test.mp4

✅ Open the video to see your model detecting:
   🚗 Cars, Trucks, Vans
   🚦 Traffic Signs
   🚥 Traffic Lights


In [12]:
from ultralytics import YOLO

print("="*80)
print("📸 TESTING MODEL ON IMAGE")
print("="*80)

# Load model
model = YOLO(r'runs\ultimate_overnight\weights\best.pt')

# Path to your image (put your image path here)
image_path = r'sign2.png'  # CHANGE THIS to your image path

print(f"\n🖼️  Testing on: {image_path}")
print("🤖 Running detection...\n")

# Run prediction
results = model.predict(
    source=image_path,
    conf=0.15,  # Low confidence to catch more objects
    save=True,
    show_labels=True,
    show_conf=True,
    project='photo_results',
    name='test_photo',
    verbose=True
)

print("\n✅ Done!")
print("📁 Result saved to: photo_results/test_photo/")
print("🎯 Open the image to see all detections!")

# Print what was detected
if len(results) > 0:
    result = results[0]
    boxes = result.boxes
    
    print(f"\n📊 Detections Summary:")
    print(f"   Total objects detected: {len(boxes)}")
    
    if len(boxes) > 0:
        print(f"\n🎯 Objects found:")
        class_counts = {}
        for box in boxes:
            class_id = int(box.cls[0])
            class_name = model.names[class_id]
            confidence = float(box.conf[0])
            
            if class_name not in class_counts:
                class_counts[class_name] = 0
            class_counts[class_name] += 1
        
        for class_name, count in class_counts.items():
            print(f"   • {class_name}: {count}")
    else:
        print("   ⚠️  No objects detected! Try lowering confidence threshold.")


📸 TESTING MODEL ON IMAGE

🖼️  Testing on: sign2.png
🤖 Running detection...



image 1/1 d:\Depi\Graduation Project\sign2.png: 576x640 1 attention_bumpers, 20.5ms
Speed: 3.3ms preprocess, 20.5ms inference, 3.3ms postprocess per image at shape (1, 3, 576, 640)
Results saved to photo_results\test_photo2



✅ Done!
📁 Result saved to: photo_results/test_photo/
🎯 Open the image to see all detections!

📊 Detections Summary:
   Total objects detected: 1

🎯 Objects found:
   • attention_bumpers: 1


In [9]:
import cv2
from ultralytics import YOLO

# Load image
img = cv2.imread('test.jpg')

# Resize to common training size
img = cv2.resize(img, (640, 640))

# Adjust brightness/contrast to match training data
img = cv2.convertScaleAbs(img, alpha=1.1, beta=10)

# Save preprocessed image
cv2.imwrite('preprocessed.jpg', img)

# Test on preprocessed image
model = YOLO('runs/ultimate_overnight/weights/best.pt')
results = model.predict('preprocessed.jpg', conf=0.1, save=True)



image 1/1 d:\Depi\Graduation Project\preprocessed.jpg: 640x640 1 Car, 33.0ms
Speed: 2.0ms preprocess, 33.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict


In [14]:
from ultralytics import YOLO

print("="*80)
print("🎥 REAL-TIME WEBCAM DETECTION")
print("="*80)

# Load model
model = YOLO(r'runs\ultimate_overnight\weights\best.pt')

print("\n📹 Starting webcam...")
print("🎯 Press 'q' to quit in the video window")
print("="*80 + "\n")

# Use YOLO's built-in predict with show=True
model.predict(
    source=0,  # 0 = webcam
    conf=0.25,
    show=True,  # This shows the window!
    stream=True,  # Stream mode for real-time
    verbose=False
)

print("\n✅ Webcam detection stopped!")


WARNING  Environment does not support cv2.imshow() or PIL Image.show()
OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'



🎥 REAL-TIME WEBCAM DETECTION

📹 Starting webcam...
🎯 Press 'q' to quit in the video window


✅ Webcam detection stopped!


In [10]:
from ultralytics import YOLO
import os
import random
import shutil

print("="*80)
print("🎯 SAMPLING 500 IMAGES FROM EACH DATASET & TESTING")
print("="*80)

# Load model
model = YOLO('Allovermodel/best.pt')

# Get all test images
test_images_dir = 'ultimate_autonomous_dataset/test/images'
all_test_images = os.listdir(test_images_dir)

# Separate by dataset
kitti_images = [img for img in all_test_images if img.startswith('kitti_')]
signs_images = [img for img in all_test_images if img.startswith('signs_')]
lights_images = [img for img in all_test_images if img.startswith('lights_')]

print(f"\n📊 Dataset Overview:")
print(f"   KITTI (Vehicles): {len(kitti_images)} total images")
print(f"   GTSRB (Signs): {len(signs_images)} total images")
print(f"   LISA (Lights): {len(lights_images)} total images")

# Sample 500 from each (or all if less than 500)
random.seed(42)  # For reproducibility

sample_kitti = random.sample(kitti_images, min(500, len(kitti_images)))
sample_signs = random.sample(signs_images, min(500, len(signs_images)))
sample_lights = random.sample(lights_images, min(500, len(lights_images)))

print(f"\n🎲 Sampling:")
print(f"   KITTI: {len(sample_kitti)} images")
print(f"   GTSRB: {len(sample_signs)} images")
print(f"   LISA: {len(sample_lights)} images")
print(f"   Total: {len(sample_kitti) + len(sample_signs) + len(sample_lights)} images")

# Create temporary folder with sampled images
sample_dir = 'temp_sample_500'
os.makedirs(sample_dir, exist_ok=True)

print(f"\n📁 Copying sampled images to {sample_dir}...")

for img in sample_kitti + sample_signs + sample_lights:
    src = os.path.join(test_images_dir, img)
    dst = os.path.join(sample_dir, img)
    shutil.copy(src, dst)

print("✅ Images copied!")

# Run predictions
print("\n🚀 Running predictions on sampled images...")
print("This will take a few minutes...\n")

results = model.predict(
    source=sample_dir,
    conf=0.6,
    save=True,
    project='proper_test_500_per_dataset',
    name='predictions',
    verbose=False
)

print("\n" + "="*80)
print("🎉 TESTING COMPLETE!")
print("="*80)
print(f"\n📊 Summary:")
print(f"   KITTI tested: {len(sample_kitti)} images")
print(f"   GTSRB tested: {len(sample_signs)} images")
print(f"   LISA tested: {len(sample_lights)} images")
print(f"   Total tested: {len(sample_kitti) + len(sample_signs) + len(sample_lights)} images")

print(f"\n📁 Results saved to:")
print(f"   proper_test_500_per_dataset/predictions/")

print(f"\n🧹 Cleaning up temporary folder...")
shutil.rmtree(sample_dir)

print("✅ Done! Check the results folder for predictions with bounding boxes!")
print("="*80)


🎯 SAMPLING 500 IMAGES FROM EACH DATASET & TESTING

📊 Dataset Overview:
   KITTI (Vehicles): 1123 total images
   GTSRB (Signs): 12630 total images
   LISA (Lights): 5430 total images

🎲 Sampling:
   KITTI: 500 images
   GTSRB: 500 images
   LISA: 500 images
   Total: 1500 images

📁 Copying sampled images to temp_sample_500...



WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs



✅ Images copied!

🚀 Running predictions on sampled images...
This will take a few minutes...



Results saved to proper_test_500_per_dataset\predictions3



🎉 TESTING COMPLETE!

📊 Summary:
   KITTI tested: 500 images
   GTSRB tested: 500 images
   LISA tested: 500 images
   Total tested: 1500 images

📁 Results saved to:
   proper_test_500_per_dataset/predictions/

🧹 Cleaning up temporary folder...
✅ Done! Check the results folder for predictions with bounding boxes!
